# GSM8K Evaluation: Baseline vs Chain-of-Thought

This notebook evaluates a language model on the GSM8K math reasoning dataset, comparing baseline performance with Chain-of-Thought (CoT) prompting.


## Setup

Install required dependencies and import libraries.


In [1]:
!pip install -q transformers datasets accelerate evaluate ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.0 MB/s eta 0:00:00


In [2]:
import re
import os
import json
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer
from datasets import load_dataset



## Data Loading

Load the GSM8K dataset and split into train, validation, and test sets.


In [3]:
dataset = load_dataset("openai/gsm8k", "main")

train_data = dataset["train"]
val_test_data = dataset["test"].shuffle(seed=42).select(range(800))

test_data = val_test_data.select(range(400))
val_data = val_test_data.select(range(400, 800))

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [4]:
# Save the dataset to a JSONL file
with open("train_data.jsonl", "w") as f:
    for sample in train_data:
        f.write(json.dumps(sample) + "\n")

with open("val_data.jsonl", "w") as f:
    for sample in val_data:
        f.write(json.dumps(sample) + "\n")

with open("test_data.jsonl", "w") as f:
    for sample in test_data:
        f.write(json.dumps(sample) + "\n")



## Model Setup

Load the Llama 3.2 1B Instruct model and configure the tokenizer.


In [7]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

pipe = pipeline(
    "text-generation",
    model=model_name,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=256,
    temperature=0.1
)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


## Evaluation Function

Define helper functions to extract numbers and evaluate model performance.


In [10]:
def extract_number(text):
    matches = re.findall(r"-?\d+(?:\.\d+)?", text)
    return float(matches[-1]) if matches else None


def evaluate_model(system_prompt: str = None):
  correct = 0
  correct_index = []
  questions, answers, generated_answers = [], [], []
  for sample in val_data:
      questions.append(sample["question"])
      answers.append(sample["answer"])


  batch_size = 4
  for i in tqdm(range(0, len(val_data), batch_size), total=len(questions)//batch_size, desc="Evaluating"):
      batch_questions = questions[i:i+batch_size]
      batch_answers = answers[i:i+batch_size]
      prompts = [f"Question: {q}\nAnswer:" for q in batch_questions]

      if system_prompt:
          prompts = [f"{system_prompt}\n\n" + p for p in prompts]

      outputs = pipe(prompts, max_new_tokens=256, batch_size=batch_size)
      for output, gt in zip(outputs, batch_answers):
          pred = output[0]["generated_text"]
          generated_answers.append(pred)

  for i, (answer, generated_answer) in enumerate(zip(answers, generated_answers)):
      gt = extract_number(answer)
      pred = extract_number(generated_answer)
      if pred == gt:
          correct += 1
          correct_index.append(1)
      else:
          correct_index.append(0)

  return correct_index, generated_answers

## Baseline Evaluation

Evaluate the model without any special prompting.


In [11]:
correct_index_base, generated_answers_base = evaluate_model()

Evaluating: 100%|██████████| 100/100 [35:42<00:00, 21.43s/it]

Accuracy: 0.18


In [14]:
accuracy = sum(correct_index_base) / len(correct_index_base)
print(f"Baseline accuracy: {accuracy:.2f}")

Baseline accuracy: 0.18


## Chain-of-Thought Evaluation

Evaluate the model with a system prompt that encourages step-by-step reasoning.


In [12]:
system_prompt = (
    "You are a helpful and accurate math assistant. "
    "For each question, reason step by step to reach the correct solution. "
    "Clearly explain your reasoning process and conclude with the final answer "
    "in the format: '#### <value>'."
)
correct_index_cot, generated_answers_cot = evaluate_model(system_prompt=system_prompt)

Evaluating: 100%|██████████| 100/100 [37:18<00:00, 22.39s/it]

Accuracy: 0.31


In [15]:
accuracy = sum(correct_index_cot) / len(correct_index_cot)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.31


In [20]:
val_data[1]

{'question': 'Matt wants cookies for a snack, but his dad wants him to eat healthily. He tells Matt he can have half as many cookies as the number of carrot sticks he eats, plus two extra for cleaning his room. If Matt wants to eat five cookies in total, how many carrot sticks does he have to eat?',
 'answer': 'First subtract the two cookies Matt gets for cleaning his room from the total number he wants to eat: 5 - 2 = <<5-2=3>>3\nThen double the number of cookies to find how many carrots sticks he has to eat: 3 * 2 = <<3*2=6>>6.\n#### 6'}

In [17]:
print(generated_answers_base[1])

Question: Matt wants cookies for a snack, but his dad wants him to eat healthily. He tells Matt he can have half as many cookies as the number of carrot sticks he eats, plus two extra for cleaning his room. If Matt wants to eat five cookies in total, how many carrot sticks does he have to eat?
Answer: 3
Answer: 3
Step 1:  Let's say the number of carrot sticks Matt eats is x.
Step 2:  Matt wants to eat half as many cookies as the number of carrot sticks he eats, so the number of cookies he wants to eat is x/2.
Step 3:  Matt also wants to eat two extra cookies for cleaning his room, so the total number of cookies he wants to eat is x/2 + 2.
Step 4:  We know that Matt wants to eat a total of five cookies, so we can set up the equation x/2 + 2 = 5.
Step 5:  To solve for x, we can subtract 2 from both sides of the equation, which gives us x/2 = 3.
Step 6:  Multiplying both sides of the equation by 2, we get x = 6.
Step 7:  Therefore, Matt has to eat 6 carrot sticks to meet his requirements.

In [18]:
print(generated_answers_cot[1])

You are a helpful and accurate math assistant. For each question, reason step by step to reach the correct solution. Clearly explain your reasoning process and conclude with the final answer in the format: '#### <value>'.

Question: Matt wants cookies for a snack, but his dad wants him to eat healthily. He tells Matt he can have half as many cookies as the number of carrot sticks he eats, plus two extra for cleaning his room. If Matt wants to eat five cookies in total, how many carrot sticks does he have to eat?
Answer: ##### <5>

## Step 1: Let's denote the number of carrot sticks Matt eats as x.
We need to establish the relationship between the number of carrot sticks eaten and the number of cookies eaten.

## Step 2: According to the problem, Matt can have half as many cookies as the number of carrot sticks he eats, plus two extra for cleaning his room.
This can be expressed as the equation: cookies = (x/2) + 2.

## Step 3: We are told that Matt wants to eat five cookies in total.
T